In [1]:
from PIL import Image
import numpy as np
import scipy.fftpack as fftpack
import imageio
from glob import glob
from ipywidgets import Image as Img
from IPython import display

In [2]:
def show_gif(filename):
    file = open(filename , "rb")
    image = file.read()
    progress= Img(
        value=image,
        format='gif',
        width=400,
        height=400)
    display.display(progress)
    
    return progress

In [3]:
def create_gif(filename, path):
    with imageio.get_writer(filename, mode='I') as writer:
        glob_path = path + '/*.png'
        for filename in sorted(glob(glob_path)):
            image = imageio.imread(filename)
            writer.append_data(image)

In [4]:
image = Image.open('E2_image.png')
image = image.resize((256,256), 1)
image = image.convert('L')

In [5]:
dctSize = image.size[0]

In [6]:
# get raw pixel values:
pixels = np.array(image.getdata(), dtype=np.float).reshape((dctSize, dctSize))

In [7]:
# perform 2-dimensional DCT (discrete cosine transform):
dct = fftpack.dct(fftpack.dct(pixels.T, norm='ortho').T, norm='ortho')

In [8]:
for i in range(0, dctSize):
    dct2 = dct.copy()

    # zero out part of the higher frequencies of the DCT values:
    dct2[i:,:] = 0
    dct2[:,i:] = 0

    # perform 2d inverse DCT to get pixels back:
    idct = fftpack.idct(fftpack.idct(dct2.T, norm='ortho').T, norm='ortho')

    # clip/convert pixel values obtained by IDCT, and create image:
    idct = idct.clip(0, 255)
    idct = idct.astype('uint8')
    img = Image.fromarray(idct)

    img.save('E2_lowpass_filter_images/img_%04d.png' % i)

In [9]:
for i in range(0, dctSize):
    dct2 = dct.copy()

    # zero out part of the lower frequencies of the DCT values:
    dct2[:i,:] = 0
    dct2[:,:i] = 0

    # perform 2d inverse DCT to get pixels back:
    idct = fftpack.idct(fftpack.idct(dct2.T, norm='ortho').T, norm='ortho')

    # clip/convert pixel values obtained by IDCT, and create image:
    idct = idct.clip(0, 255)
    idct = idct.astype("uint8")
    img = Image.fromarray(idct)

    img.save("E2_highpass_filter_images/img_%04d.png" % i)

In [10]:
create_gif('E2_lowpass_filter.gif', './E2_lowpass_filter_images')
create_gif('E2_highpass_filter.gif', './E2_highpass_filter_images')

In [11]:
lowpass_gif = show_gif('E2_lowpass_filter.gif')

A Jupyter Widget

In [12]:
lowpass_gif.close()

In [13]:
highpass_gif = show_gif('E2_highpass_filter.gif')

A Jupyter Widget

In [14]:
highpass_gif.close()